In [ ]:
import os,sys,string, time
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


## Purpose
This notebook takes the input .root file ntuples and creates .pkl files that are used for the rest of the analysis chain.<br> 
These .pkl files are much smaller and easier to deal with in python.<br> 
Which samples will be made into .pkls are defined by the Params dictionary.<br>
This notebook will need to be run at least twice to create all necessary files for the analysis chain; once for "run1" and once for "run3". Always check the names of the samples that are printed and ensure they are what you want to save.

# Setting which samples to pickle

In [ ]:
Params = {"Run":"run1", #The run number, so far either "run1" or "run3"
          "Load_lepton_signal":True, #Just e+e- samples, Majorana
          "Load_pi0_signal":False, #Just pi0 samples, Majorana
          "Load_lepton_dirac":False, #Just e+e-, Dirac
          "Load_pi0_dirac":False, #Just pi0, Dirac
          "Load_standard_bkgs":True, #Backgrounds, 'overlay', 'dirtoverlay' and 'beamoff'
          "Load_data":True, #'beamgood' sample
          "Load_DetVars":False, #overlay Detector variations
          "Load_Signal_DetVars":False, #e+e- Detector variation samples
          "Load_pi0_signal_DetVars":False, #pi0 Detector variation samples
          "Only_keep_common_DetVar_evs":True, #For DetVar samples, only keep events common between samples, True is standard
          "FLATTEN":True, #Have one row per reconstructed object in the dataframe, if True this will effectively make the nslice==1 cut.
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making UNFLATTENED pre-selection plots
          "Load_truth_vars":False, #Load the "mc_" type variables for signal, may be useful for truth studies
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":'beamgood'} 

Params, samples = Functions.new_create_sample_list(Params)
sample_locs_new = Functions.Get_all_sample_locs(Params)

print("Saving these variables: \n" + str(Params["variables_MC"]))


## Loading and pickling files

In [ ]:
common_evs = Functions.Make_common_evs(samples, sample_locs_new, Params)
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

Functions.New_load_and_pkl(samples, sample_locs_new, loc_pkls, common_evs, Params, save_str="_full_Finished")

## End of code